In [60]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from math import nan, ceil
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    page = BeautifulSoup(req.text,"html.parser")
    return page

def get_max(page):
    total = page.find("span",{"class":"vacancies-counter","data-cy":"resultCounter"}).get_text()
    total = re.search("\d+",total).group()
    total = ceil(int(total)/10)
    return total

def get_jobs(url,including=None):
    page = render_html(url)
    results = page.find("div",id="vacancyResults").find("ul")
    ref = "https://startpeople.nl"
    jobs = []
    for li in results.find_all("li",class_="vacancy-card"):
        link = ref+li.find("a")['href']
        title = li.find("h2",class_="vacancy-title").get_text()
        city = li.find("span",class_="uc-map-marker").find_previous().get_text()
        hours = li.find("span",class_="uc-clock").find_previous().get_text()
        wage = li.find("span",class_="uc-euro").find_previous().get_text()
        desc = li.find("div",class_="vacancy-description").get_text().strip()
        job = {
            "Link": link,
            "Title": title,
            "City": city,
            "Hours": hours,
            "Wage": "€"+wage,
            "Description": desc
        }
        jobs.append(job)
    if including == "max_page":
        max_page = get_max(page)
        return jobs, max_page
    else:
        return jobs

In [61]:
url = "https://startpeople.nl/en/candidate/vacancies"
jobs, max_page = get_jobs(url,including="max_page")
for i in range(2,max_page+1):
    url = f"https://startpeople.nl/en/candidate/vacancies?refinementList[language][0]=en&page={i}"
    new_jobs = get_jobs(url)
    jobs.extend(new_jobs)

In [ ]:
# results = page.find("div",id="vacancyResults").find("ul")
# ref = "https://startpeople.nl"
# jobs = []
# for li in results.find_all("li",class_="vacancy-card"):
#     link = ref+li.find("a")['href']
#     title = li.find("h2",class_="vacancy-title").get_text()
#     city = li.find("span",class_="uc-map-marker").find_previous().get_text()
#     hours = li.find("span",class_="uc-clock").find_previous().get_text()
#     wage = li.find("span",class_="uc-euro").find_previous().get_text()
#     desc = li.find("div",class_="vacancy-description").get_text().strip()
#     job = {
#         "Link": link,
#         "Title": title,
#         "City": city,
#         "Hours": hours,
#         "Wage": "€"+wage,
#         "Description": desc
#     }
#     jobs.append(job)
    # print("-"*100)
    # pprint(job,sort_dicts=False)
    # print("-"*100)
    # print(li)

In [62]:
import pandas as pd

df = pd.DataFrame(jobs)
df.to_excel("startpeople.xlsx",index=False,encoding="utf-8")